<a href="https://colab.research.google.com/github/MHX1203/DataWhale-DataMining/blob/master/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>